In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

# Seller P&L Analyis + GeoPandas 🌎

🎯 The goal of this exercise is to compute the P&L associated to each seller  

For each `seller_id` we need three items:  

- The **revenue**:
 - 10% fee on sales
 - 80 BRL per month on Olist


- The **cost**:
 - Review cost according to `{1: 100, 2: 50, 3: 40, 4: 0, 5: 0}` with review score as key and cost in BRL as value


- The **profit** made by Olist

💡 Let's not start from scratch  
❓ Import your seller training set and investigate what you already have.


In [ ]:
# YOUR CODE HERE

❓ What is missing?  
Write down a strategy to get your missing columns  
Re-use as much of what has already been coded in `seller.py` as possible

In [ ]:
# Your pseudo-code

❓ Make a copy of `seller.py`, rename it `seller_updated.py` and update it accordingly

> YOUR ANSWER HERE

❓ Compute seller profits

> YOUR ANSWER HERE

❓ Load your updated DataFrame

In [ ]:
# YOUR CODE HERE

❓ Sort sellers by profit, and analyse their profitability: conclude on a possible strategy for Olist!

In [ ]:
# YOUR CODE HERE

# Optional Bonus: GeoPandas

For any students looking for some alternative/more advanced ways to carry out and present their analysis, we can check out some GeoPandas code!

In [ ]:
!pip install geopandas

### What is [GeoPandas](https://geopandas.org/en/stable/index.html)?

GeoPandas is a Python module that adds geospatial capabilities to the popular Pandas library. 🌍 It's like a magic wand for working with geographical data, allowing you to manipulate, analyze, and visualize geospatial datasets with ease. 📊 

With GeoPandas, you can load and save various geospatial file formats:
- (Shapefile (.shp)
- GeoJSON (.geojson)
- GeoPackage (.gpkg)
- KML (.kml)
- GeoTIFF (.tif)

Once loaded, you can perform spatial operations like intersections and buffers, and create beautiful maps to showcase your findings. 🎨 It builds upon powerful geospatial libraries like Shapely and Fiona, providing a comprehensive toolset for all your geospatial needs 💪 

### Part 1: Basic usage

Similar to libraries like Seaborn, GeoPandas has its own demo datasets we can have a quick play with!

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the Natural Earth cities dataset
world_df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Print the loaded data
world_df.head()


If we want to plot our geometry out, all we have to do is call .plot()

In [ ]:
world_df.plot()

Let's use our population column to add some color by specifying `column =` in our `.plot()`

In [ ]:
world_df.plot(column  = "pop_est")

We can overlay two plots on top of each other very easily, too:

In [ ]:
# Load the Natural Earth cities dataset
cities_df = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

fig, ax = plt.subplots(figsize=(10, 6))

# Plot the world map + cities
world_df.plot(ax=ax, color='lightgray')
cities_df.plot(ax=ax, marker='o', color='red', markersize=5)

# Set our title
plt.title('Cities Overlaid on World Map')
plt.show()


If this is all you have time for, hop to the end to see an Olist example!

### Part Two: London Tube Investigation! 

First, download [this folder](https://wagon-public-datasets.s3.amazonaws.com/data-science-images/geopandas_decision_science/data.zip) and put it into a `data` folder in this directory. Let's load up a the `London_Ward.shp` shapefile (if you look in the folder you'll see it's split into a few parts - see [this](https://en.wikipedia.org/wiki/Shapefile) for why!) for London's regions and also our locations for tube stops (`london-underground.geojson`).

In [ ]:
london_stations = gpd.read_file("data/london-underground.geojson")

In [ ]:
london_map = gpd.read_file("data/London_Ward.shp")
london_map.head()

In [ ]:
london_map.plot()

Looks like we have a few too many wards. What we're more interested in is the Districts. Let's use `.dissolve(by = "DISTRICT")` to merge our District shapes together. You can think of it like a geospatial groupby!

In [ ]:
london_districts = london_map.dissolve(by = "DISTRICT", as_index = False)

In [ ]:
london_districts.plot()

That looks more reasonable! Let's overlay our tube stations

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
london_districts.plot(ax = ax)
london_stations.plot(ax = ax)

That doesn't look right 😬  This often occurs when our geometries have been saved on different projections. Let's use `.crs` to check!

🌍 Coordinate Reference Systems (CRS) in GeoPandas are like the Earth's GPS coordinates system. They provide a way to precisely locate and interpret spatial data. 🗺️ A CRS consists of a coordinate system and a datum, ensuring that geographic features are accurately represented on maps. It also allows for transformations between different CRSs, so you can project or reproject data to fit specific needs. 🔀 With CRS, GeoPandas helps you navigate the spatial world, ensuring your data aligns correctly and enabling you to explore and analyze geographic information effectively.

In [ ]:
print(london_districts.crs)
print(london_stations.crs)

Let's change the projection of our districts to match our stations and try again:

In [ ]:
london_stations = london_stations.to_crs("EPSG:27700")

In [ ]:
print(london_districts.crs)
print(london_stations.crs)

Looks like they're aligned; let's visualize to check!

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
london_districts.plot(ax = ax)
london_stations.plot(ax = ax, markersize = 2, color = "white")

Much better! Now let's use [`.buffer()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html) to see how much area is within an 800m walk of a tube station!

In [ ]:
station_buffer = london_stations.buffer(800)

What does our buffered out area look like? What's its total area?

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
london_districts.plot(ax = ax)
station_buffer.plot(ax = ax, color = "white")

To find its area we can simply use `.unary_union` (which blends all of our shapes together) and then `.area`

In [ ]:
print(f"Total area within 800m of a station is: {station_buffer.unary_union.area} square meters \
or {station_buffer.unary_union.area / 1_000_000} square km")

### Finally, we can look at which areas are best and worst served!

First we need to dissolve our buffer geometries all into one large shape. But we can only call use the `.dissolve()` method on a GeoDataFrame and our `station_buffer` is a `GeoSeries`:

In [ ]:
type(station_buffer)

So first we convert to a `GeoDataFrame` (by simply writing `GeoDataFrame(<our_series>)`) and then we can call `.dissolve()`

N.B. When converting to a GeoDataFrame, we also have to specify that our geometry is in our first - and only column - by writing `geometry = 0`

In [ ]:
buffer_df = gpd.GeoDataFrame(station_buffer, geometry=0).dissolve()

In [ ]:
buffer_df

Our final step is to use the `.overlay()` function with the `"intersection"` argument. This will return to us the intersection between our two GeoDataFrames on a district by district basis! Then create our final column in our DataFrame as the area of the geometry.

<img src = "https://wagon-public-datasets.s3.amazonaws.com/data-science-images/gpd_overlays.webp">

In [ ]:
gdf_overlap = gpd.overlay(buffer_df,london_districts, how='intersection')

Now, we can plot out one district!

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
london_districts.plot(ax = ax)
# Here we're just selecting the geometry with our second district (Camden)
gdf_overlap.loc[[1],'geometry'].plot(ax = ax,color = "white")
plt.title('Camden tube coverage')
fig.show()

All that's left is to create a new column which describes the area of each of our polygons so we can say how much of each DISTRICT was covered by our tubes! Creating new columns works just like it does in Pandas and all we need to do is access the `.area` attribute of our column.

In [ ]:
gdf_overlap["area_covered"] = gdf_overlap["geometry"].area

Then we merge it all into one GeoDataFrame

In [ ]:
merged = london_districts.merge(gdf_overlap[["DISTRICT", "area_covered"]], on = "DISTRICT")

In [ ]:
merged.head()

Calculate the overall area of each district:

In [ ]:
merged["total_area"] = merged.geometry.area

And divide!

In [ ]:
merged["percentage_covered"] = (merged["area_covered"] / merged["total_area"]) * 100

In [ ]:
merged[["DISTRICT", "percentage_covered"]].sort_values(by = "percentage_covered")

## How can we apply this to Olist? 

Well, for starters we can make some great plots. Run the cells below to generate a map of Brazil with NPS overlaid. Where you go from here is your choice tomorrow 💪

In [ ]:
import geopandas as gpd
from olist.data import Olist
from olist.order import Order
import matplotlib.pyplot as plt
data = Olist().get_data()
orders = Order().get_training_data()

In [ ]:
# Calculate NPS score per state
merge = data['orders'].merge(data['order_reviews'], on='order_id')\
                      .merge(data['customers'], on='customer_id')

by_state_nps = merge.groupby(['customer_state'], as_index=False)['review_score'] \
                    .apply(lambda s: s.map({5:1, 4:0, 3:-1, 2:-1, 1:-1}).sum() / s.count()) \
                    .rename(columns={"review_score":"average_nps"})

nps_brazil = orders.review_score.map({5:1, 4:0, 3:-1, 2:-1, 1:-1}).sum() / orders.review_score.count()

# Preprocess GeoDataFrame
brazil = gpd.read_file('data/brazil.gpkg')
brazil.rename({"sigla": 'customer_state'}, axis=1, inplace=True)
brazil_nps = brazil.merge(by_state_nps, on='customer_state', how='left')
brazil_nps = brazil_nps[['customer_state', 'average_nps', 'geometry']]
brazil_nps['center_x'] = brazil_nps['geometry'].map(lambda c: c.centroid.x)
brazil_nps['center_y'] = brazil_nps['geometry'].map(lambda c: c.centroid.y)

# Plot figure
fig, ax = plt.subplots(figsize=(11,7))
brazil_nps.plot(column = "average_nps",
                   cmap='RdBu',
                   legend=True,
                   edgecolor='black', 
                   ax=ax,
                   label='Average nps per state')

# Labeling and styling the plot
ax.set_xlabel("Latitude", labelpad=10)
ax.set_ylabel("Longitude", labelpad=10)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])

ax.set_title('Average net promoter score (NPS per state)', pad=10, size=16)

ax.text(-48,-32,f'Brazil NPS Score = {nps_brazil*100:.1f}%', size=12)

#Annotating each state's name on to the centre of the state's geometry
for idx, row in brazil_nps.iterrows():
    ax.text(row["center_x"], row["center_y"],row["customer_state"], ha="center", va="center")
    
plt.show();